In [2]:
import sys
sys.path.append('/home/yanrui/code/CLIPBased_TAD')

import random
import os.path
from losses.clip import clip_loss2
from model.CLIPTAD.CLIPModel import CLIPModel
import os
import torch
import heapq  # 用于管理最小堆
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import logging
from torch.utils.data import DataLoader
from tqdm import tqdm
from dataset.train.wwadl import detection_collate
from dataset.train.wwadl_multi import WWADLDatasetMutiAll
from configs.config import config 

GLOBAL_SEED = 42


def _to_var(data: dict, device):
    for key, value in data.items():
        data[key] = value.to(device)  # Directly move tensor to device
    return data

def worker_init_fn(worker_id):
    np.random.seed(GLOBAL_SEED + worker_id)

logger = logging.getLogger(__name__)

class BestModelSaver:
    def __init__(self, check_point_path, max_models=10):
        self.check_point_path = check_point_path
        self.max_models = max_models
        # 使用最大堆保存模型信息 [(负的metric, model_path), ...]
        self.best_models = []

    def save_model(self, model_state_dict, model_name, metric, is_save=False):
        # 构造保存路径
        model_path = os.path.join(self.check_point_path, f"{model_name}.pt")

        if is_save:
            torch.save(model_state_dict, model_path)
            return

        # 如果队列未满，直接保存模型
        if len(self.best_models) < self.max_models:
            torch.save(model_state_dict, model_path)
            # 保存负的metric以构造最大堆
            heapq.heappush(self.best_models, (-metric, model_path))
            print(f"Model saved: {model_path} (Metric: {metric:.5f})")
        else:
            # 检查是否优于当前最差模型（堆顶是负的最大值，对应正的最小值）
            if metric < -self.best_models[0][0]:  # 假设指标是损失，越小越好
                # 删除最差模型
                _, worst_model_path = heapq.heappop(self.best_models)
                if os.path.exists(worst_model_path):
                    os.remove(worst_model_path)
                    print(f"Old model removed: {worst_model_path}")

                # 保存新模型
                torch.save(model_state_dict, model_path)
                heapq.heappush(self.best_models, (-metric, model_path))
                print(f"Model saved: {model_path} (Metric: {metric:.5f})")
            else:
                print(f"Model not saved. Metric: {metric:.5f} is worse than the top 10.")

    def get_best_models(self):
        # 返回按指标从小到大排序的模型列表（还原负的metric）
        return sorted([(-metric, path) for metric, path in self.best_models], key=lambda x: x[0])

class Trainer_ForCLIP(object):
    def __init__(self,
                 train_dataset,
                 model
                 ):
        super(Trainer_ForCLIP, self).__init__()
        self.model = model
        self.train_dataset = train_dataset

        self.batch_size = 4
        self.num_epoch = 300
        self.lr_rate_adjust_epoch = 30
        self.lr_rate_adjust_factor = 0.5
        # loss setting -----------------------------------------------------------
        self.loss = clip_loss2

        # learning config ---------------------------------------------------------
        self.opt_method = 'adamw'
        self.lr_rate = 4e-4
        self.weight_decay = 1e-3

        self.check_point_path = ' /home/yanrui/code/CLIPBased_TAD/model/CLIPTAD/preTrain_Emb/'

        self.model_info = 'clipEmbedding'
        self.writer = SummaryWriter(os.path.join(self.check_point_path, f'tb_{self.model_info}'))

        # DDP setting -------------------------------------------------------------
        self.dist_url = 'env://'
        self.rank = 0
        self.world_size = 0
        self.gpu=0

        self.device = 'cuda'
    
    def _init_optimizer(self):

        params = self.model.parameters()

        if self.opt_method == 'adam':
            self.optimizer = torch.optim.Adam(params=params,
                                              lr=self.lr_rate,
                                              weight_decay=self.weight_decay)
        elif self.opt_method == 'adamw':
            self.optimizer = torch.optim.AdamW(params=params,
                                               lr=self.lr_rate,
                                               weight_decay=self.weight_decay)
        else:
            self.optimizer = torch.optim.SGD(params=params,
                                             lr=self.lr_rate,
                                             weight_decay=self.weight_decay)

        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer,
                                                         self.lr_rate_adjust_epoch,
                                                         self.lr_rate_adjust_factor)

    def _train_one_step(self, data, targets):

        data = _to_var(data, self.device) # 将本来是字典的数据移动到设备上

        # data = data.to(self.device)  # 确保输入数据在正确的设备上
        targets = [t.to(self.device) for t in targets] # 将label移动到设备上
        self.optimizer.zero_grad()


        try:
            sig_emb, text_emb = self.model(data, targets)
            tmp_sig_emb, tmp_text_emb = sig_emb, text_emb
            
        except AssertionError as e:
            logging.info("Error occurred during training, saving data and model parameters for debugging...")

            # 创建保存路径
            error_save_path = os.path.join(self.check_point_path, "debug")
            os.makedirs(error_save_path, exist_ok=True)

            # 保存导致问题的输入数据
            data_save_path = os.path.join(error_save_path, "error_data.pt")
            torch.save(data, data_save_path)
            logging.info(f"Input data saved to: {data_save_path}")

            # 保存模型参数
            model_save_path = os.path.join(error_save_path, "error_model.pth")
            torch.save(self.model.state_dict(), model_save_path)
            logging.info(f"Model parameters saved to: {model_save_path}")

            # 再次抛出异常以中断训练
            raise e

        loss = self.loss(tmp_sig_emb, tmp_text_emb)

        # 反向传播
        loss.backward(retain_graph=True)

        # 优化器更新权重
        self.optimizer.step()

        # 无需分布式同步，直接返回损失
        return loss.item()
    
    def set_seed(self, seed):
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.cuda.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.backends.cudnn.deterministic =True

    def training(self):
        # 给不同的进程分配不同的、固定的随机数种子
        self.set_seed(2024)
        # register_hooks(self.model) # 钩子函数？不懂，后面去研究
        device = torch.device(self.device)

        # dataset loader -------------------------------------------------------------------------------
        nw = min([os.cpu_count(), self.batch_size if self.batch_size > 1 else 0, 8])  # number of workers
        print('Using {} dataloader workers every process'.format(nw))

        train_loader = DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,  # DataParallel 不需要使用分布式采样
            pin_memory=True,
            num_workers=nw,  # 动态设置 workers
            collate_fn=detection_collate,  # 自定义 collate_fn
            worker_init_fn=worker_init_fn,  # 初始化每个 worker 的随机种子
            drop_last = True
        )

        # load model ------------------------------------------------------------------------------------

        # 转为DataParallel模型 ---------------------------------------------------------------------------
        if torch.cuda.device_count() > 1:
            print(f"Using {torch.cuda.device_count()} GPUs for training")
            self.model = torch.nn.DataParallel(self.model, device_ids=list(range(torch.cuda.device_count())))
        else:
            print("Using a single GPU for training")

        self.model = self.model.to(device=device)

        self._init_optimizer()

        mini_train_loss = float('inf')
        saver = BestModelSaver(self.check_point_path, max_models=1)  # 初始化最佳模型管理

        for epoch in range(self.num_epoch):
            np.random.seed(epoch)  # 设置随机种子
            self.model.train()

            tbar = tqdm(train_loader)

            iteration = 0
            cost_val = 0
            loss_val = 0

            for clips, targets in tbar:
                iteration += 1
                
                loss = self._train_one_step(clips, targets)

                loss_val += loss
                cost_val += loss

                tbar.set_description('Epoch: %d: ' % (epoch + 1))
                tbar.set_postfix(train_loss=loss)
                # 每次迭代清理显存
                torch.cuda.empty_cache()


            tbar.close()

            loss_val /= (iteration + 1)
            cost_val /= (iteration + 1)
            plog = 'Epoch-{} Loss: Total - {:.5f}, loc - {:.5f}' \
                .format(epoch, cost_val, loss_val)

            logging.info(plog)

            self.scheduler.step()

            # 保存当前模型
            saver.save_model(self.model.state_dict(), f"{self.model_info}-epoch-{epoch}", cost_val)

            self.writer.add_scalar("Train Loss", cost_val, epoch)
            self.writer.add_scalar("loss_val Loss", loss_val, epoch)
    


In [6]:
# 安全加载函数
def safe_load(model, pretrained_path):
    pretrained_dict = torch.load(pretrained_path)
    model_dict = model.state_dict()
    
    # 1. 过滤不匹配的键
    pretrained_dict = {k: v for k, v in pretrained_dict.items() 
                      if k in model_dict and v.shape == model_dict[k].shape}
    
    # 2. 严格检查版本
    for k, v in pretrained_dict.items():
        if model_dict[k].requires_grad:
            v.requires_grad_(True)  # 保持梯度追踪一致性
    
    model.load_state_dict(pretrained_dict, strict=False)  # 非严格模式
    print(f"Loaded {len(pretrained_dict)}/{len(model_dict)} parameters")
    return model

In [7]:
import os 
os.environ['CUDA_VISIBLE_DEVICES']='0' 

if __name__ == '__main__':
    #-----import data
    train_dataset_multi = WWADLDatasetMutiAll(config['path']['train_data_path'], split='train')
    #-----import model
    model = CLIPModel().cuda()
    model = safe_load(model, '/home/yanrui/code/CLIPBased_TAD/model/CLIPTAD/preTrain_Emb/clip_model_state_dict_epoch_200.pth')
    #-----train
    trainer = Trainer_ForCLIP(train_dataset_multi, model)
    torch.autograd.set_detect_anomaly(True)
    trainer.training()

Loaded 467/467 parameters
Using 4 dataloader workers every process
Using a single GPU for training


  0%|          | 0/2433 [00:00<?, ?it/s]/home/yanrui/anaconda3/envs/py38/lib/python3.8/site-packages/torch/autograd/__init__.py:197: UserWarning: Error detected in MmBackward0. Traceback of forward call that caused the error:
  File "/home/yanrui/anaconda3/envs/py38/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/yanrui/anaconda3/envs/py38/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/yanrui/anaconda3/envs/py38/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/yanrui/anaconda3/envs/py38/lib/python3.8/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/yanrui/anaconda3/envs/py38/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 619, in start
    self.io_loop.start()
  File "/home/yanrui/anaconda3/envs/py38/lib/python3.8/site-packages/tornado/plat

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor [512, 512]], which is output 0 of AsStridedBackward0, is at version 2; expected version 1 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!

In [1]:
import sys
sys.path.append('/home/yanrui/code/CLIPBased_TAD')

import torch
from configs.config import config 
from model.CLIPTAD.CLIPTAD import CLIPTAD
from model.models import RFCLIP 
from dataset.train.wwadl_multi import WWADLDatasetMutiAll
from trains.train import Trainer


/home/yanrui/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os 
os.environ['CUDA_VISIBLE_DEVICES']='1' 

if __name__ == '__main__':
    #------import data
    train_dataset_multi = WWADLDatasetMutiAll(config['path']['train_data_path'], split='train')
    #------import model
    model = RFCLIP(config)
    #------train
    trainer = Trainer(config, train_dataset_multi, model)
    trainer.training()

TypeError: __init__() got multiple values for argument 'isTrainClip'

In [1]:
import sys
sys.path.append('/home/yanrui/code/WiFi_TAD/WiFiTAD_main')

